# 🎶 Objectif 2 : Analyse des Flux Musicaux

#### Rappel

Analyse des aires de flux pour quatre caractéristiques musicales clé. L'objectif est d'identifier les périodes où il serait stratégique de sortir des morceaux qui se démarquent par leur niveau de dansabilité, d'énergie, d'acoustique, ou d'équilibre.

In [1]:
# Import
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go

In [2]:
# Chargement
df = pd.read_csv("./dataset/dataset_filtered.csv")

# Agrégation des données par année pour les quatre caractéristiques musicales
df_agg = df.groupby(['released_year', 'released_month'])[['danceability', 'energy', 'acousticness', 'valence']].mean().reset_index()

In [3]:
# Initialisation de l'application Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Évolution des caractéristiques cusicales au fil des années"),

    # Graphique en aires de flux
    dcc.Graph(
        id='stacked-area-chart',
        config={'displayModeBar': False}
    ),

    # Slider pour sélectionner une période
    html.Div("Sélectionnez une période pour :", style={'margin-top': '20px'}),
    dcc.RangeSlider(
        id='year-slider',
        min=df_agg['released_year'].min(),
        max=df_agg['released_year'].max(),
        step=1,
        marks={str(year): str(year) for year in range(df_agg['released_year'].min(), df_agg['released_year'].max() + 1, 5)},
        value=[df_agg['released_year'].min(), df_agg['released_year'].max()]
    )
])

In [4]:
# Callback pour mettre à jour le graphique en fonction de la période sélectionnée
@app.callback(
    Output('stacked-area-chart', 'figure'),
    Input('year-slider', 'value')
)
def update_graph(selected_years):
    # Filtrer les données pour la période sélectionnée
    filtered_df = df_agg[(df_agg['released_year'] >= selected_years[0]) & (df_agg['released_year'] <= selected_years[1])]

    fig = go.Figure()

    # Ajout des couches d'aire de flux pour chaque caractéristique
    fig.add_trace(go.Scatter(x=filtered_df['released_year'], y=filtered_df['danceability'],
                             mode='lines', stackgroup='one', name='Dansabilité', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=filtered_df['released_year'], y=filtered_df['energy'],
                             mode='lines', stackgroup='one', name='Énergie', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=filtered_df['released_year'], y=filtered_df['acousticness'],
                             mode='lines', stackgroup='one', name='Acoustique', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=filtered_df['released_year'], y=filtered_df['valence'],
                             mode='lines', stackgroup='one', name='Valence', line=dict(color='orange')))

    # Ajout des points pour afficher la moyenne mensuelle de chaque caractéristique
    colors = {
        'danceability': 'blue',
        'energy': 'red',
        'acousticness': 'green',
        'valence': 'orange'
    }

    for feature in ['danceability', 'energy', 'acousticness', 'valence']:
        monthly_avg = df.groupby(['released_year', 'released_month'])[feature].mean().reset_index()
        monthly_avg_filtered = monthly_avg[(monthly_avg['released_year'] >= selected_years[0]) & (monthly_avg['released_year'] <= selected_years[1])]

        fig.add_trace(go.Scatter(
            x=monthly_avg_filtered['released_year'] + monthly_avg_filtered['released_month'] / 12,  # Position des mois sur l'axe X
            y=monthly_avg_filtered[feature],
            mode='markers',  # Marquer les points
            name=f'{feature} (Moyenne Mensuelle)',
            marker=dict(size=8, color=colors[feature], symbol='circle')
        ))

    # Mise en forme du graphique
    fig.update_layout(
        title="Évolution de la Dansabilité, Énergie, Acoustique et Valence des Musiques",
        xaxis_title="Année",
        yaxis_title="Moyenne des attributs",
        legend_title="Caractéristiques",
        template="plotly_white"
    )

    return fig

In [5]:
# Exécution du serveur (http://127.0.0.1:8050/)
if __name__ == '__main__':
    app.run_server(debug=True,port=8051)